## Sparse solutions with L1 regularization

In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from io import StringIO
from sklearn.impute import SimpleImputer
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from itertools import combinations
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

# assign data to X, Y
X = df_wine.iloc[:130, 1:].values
y = df_wine.iloc[:130, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# Standardization
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [16]:
lamda = 40

lr = LogisticRegression(penalty='l1', C=1.0/lamda, solver='liblinear')
lr.fit(X_train_std, y_train)

print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_test_std, y_test), '\n')
print(lr.coef_)

Training accuracy: 0.9326923076923077
Test accuracy: 0.8846153846153846 

[[ 0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
  -0.1784056]]


## Sequential feature selection algorithms

In [ ]:
class SBS():
  def __init__(self, estimator, k_features, test_size):
    self.estimator = clone(estimator)     # e.g: knn
    self.k_features = k_features          # desired number of features
    self.test_size = test_size            # default: 25% data in the test set

  def fit(self, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size, random_state = 1)           # split data to train set and test set, random_state makes sure the spliting is fixed in every run. 
    dim = X_train.shape[1]                                                            # number of features of the original data set (13)
    self.indices_ = tuple(range(dim))                                                 # the column indices of the fnal feature subset (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
    self.subsets_ = [self.indices_]                                                   # [(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)]
    score = self._calc_score(X_train, y_train, X_test, y_test, self.indices_)         # score of model have all features
    self.scores_ = [score]                                                            # score of model have all features
  
    while dim > self.k_features:
      scores = []
      subsets = []
      for p in combinations(self.indices_, r=dim - 1):                # (r-1) subsets of r features, where number of features of each subset is (r-1)
        score = self._calc_score(X_train, y_train,X_test, y_test, p)  # calculate score of a subset
        scores.append(score)                                          # array of (r-1) score of (r-1) subsets, e.g: [0.3, 0.5, 0.9, 0.8, 0.99]
        subsets.append(p)                                             # array of (r-1) subsets of r features, e.g: [(0,1), (0,2), (1,2)]
      best = np.argmax(scores)                                        # return the index of the greatest score of a combination, e.g: 9
      self.indices_ = subsets[best]                                   # return best subset and overwrite in indices_, e.g: (1,2)
      self.subsets_.append(self.indices_)                             # append best subset to array subsets_, e.g: [(1,2,3,4), (1,3,4), (1,3)]
      dim -= 1                                                        # dim = dim - 1
      self.scores_.append(scores[best])                               # append score of best subsets to array scores_
    return self

  def _calc_score(self, X_train, y_train, X_test, y_test, indices):
    self.estimator.fit(X_train[:, indices], y_train)
    y_pred = self.estimator.predict(X_test[:, indices])
    score = accuracy_score(y_test, y_pred)
    return score

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)                   # object knn with n_neighbors = 5
sbs = SBS(estimator=knn, k_features=1, test_size=0.25)      # object sbs
sbs.fit(X_train_std, y_train)                               # train model, return subsets_, scores_
sbs.subsets_

In [ ]:
k_feat = [len(k) for k in sbs.subsets_]
plt.plot(k_feat, sbs.scores_, marker='o')
plt.ylim([0.7, 1.01])
plt.ylabel('Accuracy')
plt.xlabel('Number of features')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
d_k = 7

k = list(sbs.subsets_[13-d_k])
print('selected features: ', df_wine.columns[1:][k])

knn.fit(X_train_std, y_train)
print('\nTraining accuracy of all features:', knn.score(X_train_std, y_train))
print('Test accuracy of all features:', knn.score(X_test_std, y_test))

knn.fit(X_train_std[:, k], y_train)
print('\nTraining accuracy of' , d_k , 'features :', knn.score(X_train_std[:, k], y_train))
print('Test accuracy of', d_k , 'features :', knn.score(X_test_std[:, k], y_test))

## Assessing feature importance with random forests

In [ ]:
feat_labels = df_wine.columns[1:]
forest = RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
  print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

importances
indices

In [ ]:
plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), importances[indices], align='center')
plt.xticks(range(X_train.shape[1]), feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()